In [59]:
import Pkg;
Pkg.add("DataStructures")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


## Introduction
Goal of this paper is to write and test following algorithms
1. DFS, BFS
2. Topological Sorting
3. Decomposition to Strongly connected components
4. Checking if graph is bipartite.

## Testing
Tests are performed on data in `aod_testy1`
Every graph in file `g**-i.txt` has got graph with roughly $|V|=10^i$ vertices and $|E| \leq 3|V|$

### Input Format
File with following lines
- letter `D` or `U` specifying if graph is directed or undirected
- $n$ - amount of vertices, we assume that $V = \{1,2,...,n\}$
- $m$ - amount of edges
- $m$ edges in format `u v`

In [1]:
function readgraph(filepath)::Array{Array{Int64, 1}, 1}
    n = 0
    graph = []
    
    open(filepath) do f
        is_undirected = readline(f) == "U"
        n = parse(Int, readline(f))
            
        for i in 1:n
            push!(graph, [])
        end
        
        m = parse(Int, readline(f))
                
        for _ in 1:m
            u, v = map(x -> parse(Int, x), split(readline(f), " "))
            push!(graph[u],v)
            if is_undirected
                push!(graph[v], u) 
            end
        end

    end
    return graph
end

readgraph (generic function with 1 method)

# DFS, BFS
Algorithms should print out order of visiting vertices and search tree.

In [51]:
using DataStructures

function dfs(graph, start)
    predecessors = Array{Int, 1}(undef, length(graph))
    fill!(predecessors, -1)
    order = Array{Int, 1}(undef, length(graph))
    fill!(order, -1)
    order_index = 1
    stack = Stack{Int}()
    push!(stack, start)
    visited = Array{Bool, 1}(undef, length(graph))
    fill!(visited, false)
    for i in 1:length(graph)
        if visited[i]
            continue
        end
        
        visited[i] = true
        if(isempty(stack))
            push!(stack, i)
        end
        
        
        while !isempty(stack)
            curr = pop!(stack)

            order[order_index] = curr
            order_index += 1

            for neighbour in graph[curr]
                if !visited[neighbour]
                    push!(stack, neighbour)
                    visited[neighbour] = true
                    predecessors[neighbour] = curr
                end
            end
        end
    end
    return order, predecessors
end

function bfs(graph, start)
    predecessors = Array{Int, 1}(undef, length(graph))
    fill!(predecessors, -1)
    order = Array{Int, 1}(undef, length(graph))
    fill!(order, -1)
    order_index = 1
    queue = Queue{Int}()
    enqueue!(queue, start)
    visited = Array{Bool, 1}(undef, length(graph))
    fill!(visited, false)
    for i in 1:length(graph)
        if visited[i]
            continue
        end
        
        visited[i] = true
        if(isempty(queue))
            enqueue!(queue, i)
        end
        
                    
        while !isempty(queue)
            curr = first(queue)
            dequeue!(queue)

            order[order_index] = curr
            order_index += 1

            for neighbour in graph[curr]
                if !visited[neighbour]
                    enqueue!(queue, neighbour)
                    visited[neighbour] = true
                    predecessors[neighbour] = curr
                end
            end
        end
    end
    return order, predecessors
end


function test_search(algorithm, graph_filename, start, print_tree)
    graph = readgraph(graph_filename)
    order, predecessors = algorithm(graph, start)
    println(graph_filename)
    println("Visiting Order: $(order)")
    if print_tree
        println("Search Tree: $(predecessors)")
    end  
end

test_search (generic function with 1 method)

In [64]:
for algorithm in [bfs, dfs]
    println(algorithm)
    for letter in ['a', 'b']
        for directed in ['u', 'd']
            filename = string("aod_testy1/1/g1", letter, "-", directed, ".txt")
            test_search(bfs, filename, 1, true)
        end
    end
    println()
end


bfs
aod_testy1/1/g1a-u.txt
Visiting Order: [1, 3, 2, 5, 6, 4]
Search Tree: [-1, 1, 1, 2, 3, 3]
aod_testy1/1/g1a-d.txt
Visiting Order: [1, 3, 2, 5, 6, 4]
Search Tree: [-1, 1, 1, 2, 3, 3]
aod_testy1/1/g1b-u.txt
Visiting Order: [1, 2, 4, 3, 6, 8, 5, 7]
Search Tree: [-1, 1, 2, 1, 6, 2, 6, 4]
aod_testy1/1/g1b-d.txt
Visiting Order: [1, 2, 4, 3, 6, 8, 5, 7]
Search Tree: [-1, 1, 2, 1, 6, 2, 6, 4]

dfs
aod_testy1/1/g1a-u.txt
Visiting Order: [1, 3, 2, 5, 6, 4]
Search Tree: [-1, 1, 1, 2, 3, 3]
aod_testy1/1/g1a-d.txt
Visiting Order: [1, 3, 2, 5, 6, 4]
Search Tree: [-1, 1, 1, 2, 3, 3]
aod_testy1/1/g1b-u.txt
Visiting Order: [1, 2, 4, 3, 6, 8, 5, 7]
Search Tree: [-1, 1, 2, 1, 6, 2, 6, 4]
aod_testy1/1/g1b-d.txt
Visiting Order: [1, 2, 4, 3, 6, 8, 5, 7]
Search Tree: [-1, 1, 2, 1, 6, 2, 6, 4]



# Topological Ordering
Function should print appropriate information if graph cannot be topologically sorted.  
If $n \leq 200$ then function should print out topologically sorted array of vertices

In [58]:
using DataStructures

function topological_ordering(graph)
    order = Stack{Int}()
    
    indegree = Array{Int, 1}(undef, length(graph))
    fill!(indegree, 0)
    
    for invertices in graph
        for invertice in invertices
            indegree[invertice] += 1
        end
    end
    
    stack = Stack{Int}()
    for i in 1:length(graph)
        if indegree[i] == 0
            push!(stack, i)
        end
    end
    while !isempty(stack)
        node = pop!(stack)
        push!(order, node)
        
        for neighbour in graph[node]
            indegree[neighbour] -= 1
            if indegree[neighbour] == 0
                push!(stack, neighbour)
            end
        end
    end
    return order
end

function test_topological_ordering(graph)
    order = topological_ordering(graph)
    
    if length(order) < length(graph)
        println("Graph cannot be topologically sorted.")
    else 
        println("Graph can be topologically sorted")
        if (length(graph) <= 200)
            println(order)
        end
    end
end


test_topological_ordering (generic function with 1 method)

In [59]:
for filetype in ['a', 'b']
    println("TYPE ", filetype)
    for i in 1:6
        filename = string("aod_testy1/2/g2",filetype,"-",i,".txt")
        println(filename)
        graph = readgraph(filename)
        test_topological_ordering(graph)
    end
end

TYPE a
aod_testy1/2/g2a-1.txt
Graph can be topologically sorted
Stack{Int64}(Deque [[1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]])
aod_testy1/2/g2a-2.txt
Graph can be topologically sorted
Stack{Int64}(Deque [[1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 2, 12, 22, 32, 42, 52, 62, 72, 82, 92, 3, 13, 23, 33, 43, 53, 63, 73, 83, 93, 4, 14, 24, 34, 44, 54, 64, 74, 84, 94, 5, 15, 25, 35, 45, 55, 65, 75, 85, 95, 6, 16, 26, 36, 46, 56, 66, 76, 86, 96, 7, 17, 27, 37, 47, 57, 67, 77, 87, 97, 8, 18, 28, 38, 48, 58, 68, 78, 88, 98, 9, 19, 29, 39, 49, 59, 69, 79, 89, 99, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]])
aod_testy1/2/g2a-3.txt
Graph can be topologically sorted
aod_testy1/2/g2a-4.txt
Graph can be topologically sorted
aod_testy1/2/g2a-5.txt
Graph can be topologically sorted
aod_testy1/2/g2a-6.txt
Graph can be topologically sorted
TYPE b
aod_testy1/2/g2b-1.txt
Graph cannot be topologically sorted.
aod_testy1/2/g2b-2.txt
Graph cannot be topologically sorted.
aod_testy1/2/g2b-3.txt
Graph 

# Strongly Connected Components
Algorithm will consist of 3 steps.
1. get poststack of `G`
2. Get transposition of a graph `G`
3. for every vertex try to get it's component

In [82]:
function getpoststack(graph)
    visited = Array{Bool, 1}(undef, length(graph))
    fill!(visited, false)
    finished = Array{Bool, 1}(undef, length(graph))
    fill!(finished, false)
    stack = Stack{Int}()
    post = Stack{Int}()

    for i in 1:length(graph)
        if visited[i]
            continue
        end
        
        push!(stack, i)
        while !isempty(stack)
            curr = pop!(stack)

            if !visited[curr]
                visited[curr] = true
                push!(stack, curr)
                for neighbour in graph[curr]
                    if !visited[neighbour]
                        push!(stack, neighbour)
                    end
                end
            else
                if !finished[curr]
                    push!(post, curr)
                    finished[curr] = true
                end
            end
        end
    end
    return post
end

function getreverse(graph)
    reversed = []
    for i in 1:length(graph)
        push!(reversed, [])
    end
    
    for i in 1:length(graph)
        for target in graph[i]
            push!(reversed[target], i)
        end
    end
    return reversed
end

function getisland(graph, start, visited)
    component = []
    stack = Stack{Int}()
    push!(stack, start)
    visited[start] = true
    
    while(!isempty(stack))
        curr = pop!(stack)
        push!(component, curr)
        
        for neighbour in graph[curr]
            if (!visited[neighbour])
                visited[neighbour] = true
                push!(stack, neighbour)
            end
        end
    end
    return component
end

function stronglyconnectedcomponents(graph)
    poststack = getpoststack(graph)
    reversed = getreverse(graph)
    components = []
    
    visited = Array{Bool, 1}(undef, length(graph))
    fill!(visited, false)
    
    while(!isempty(poststack))
        currstart = pop!(poststack)
        if visited[currstart]
            continue
        end
        
        component = getisland(reversed, currstart, visited)
        push!(components, component)
        
    end
    return components
end

function teststronglyconnectedcomponents(filename)
    graph = readgraph(filename)
    components = stronglyconnectedcomponents(graph)
    println(string("graph size: ", length(graph), " ssc amount: ", length(components)))
    for i in 1:length(components)
        print(string("Component ", i, " size: ", length(components[i])))
        if length(graph) <= 200
            print(": ")
            print(components[i])
        end
        println()
    end
end

teststronglyconnectedcomponents (generic function with 1 method)

In [83]:
for i in 1:6
    filename = string("aod_testy1/3/g3-", i, ".txt")
    println(filename)
    teststronglyconnectedcomponents(filename)
    println()
end


aod_testy1/3/g3-1.txt
graph size: 16 ssc amount: 5
Component 1 size: 5: Any[1, 5, 4, 3, 2]
Component 2 size: 4: Any[12, 15, 14, 13]
Component 3 size: 4: Any[6, 8, 9, 7]
Component 4 size: 2: Any[10, 11]
Component 5 size: 1: Any[16]

aod_testy1/3/g3-2.txt
graph size: 107 ssc amount: 5
Component 1 size: 6: Any[1, 6, 5, 4, 3, 2]
Component 2 size: 40: Any[67, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68]
Component 3 size: 36: Any[7, 13, 19, 25, 31, 37, 38, 39, 40, 41, 42, 36, 35, 29, 28, 34, 22, 21, 27, 33, 15, 9, 8, 16, 10, 23, 17, 11, 30, 24, 18, 12, 32, 26, 20, 14]
Component 4 size: 24: Any[43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]
Component 5 size: 1: Any[107]

aod_testy1/3/g3-3.txt
graph size: 1008 ssc amount: 5
Component 1 size: 7
Component 2 size: 400
Component 3 size: 400
Component 4 size: 200
Component 5 size: 1



In [98]:
function colorisland(graph, colors, start)
    stack = Stack{Int}()
    push!(stack, start)
    
    while(!isempty(stack))
        curr = pop!(stack)
        
        my_color = colors[curr]
        enemy_color = 0
        if my_color == 0
            enemy_color = 1
        end
        
        for neighbour in graph[curr]
            if (colors[neighbour] == -1)
                colors[neighbour] = enemy_color
                push!(stack, neighbour)
            elseif (colors[neighbour] == my_color)
                return true
            end
        end
    end
    return false
end

function isbipartite(graph)
    colors = Array{Int, 1}(undef, length(graph))
    fill!(colors, -1)
    
    iserror = false
    for i in 1:length(graph)
        if colors[i] == -1
            iserror = colorisland(graph, colors, i)
            if iserror
                break
            end
        end
    end
    return iserror, colors
end

function printbipartite(colors)
    for color in 0:1
        println("Color: ", color)
        for i in 1:length(colors)
            if colors[i] == color
                print(string(i, " "))
            end
        end
        println()
    end
        
end

function testisbipartite(filename)
    graph = readgraph(filename)
    iserror, colors = isbipartite(graph)
    println(string("filename: ",filename ," size: ", length(graph)))
    if iserror
        println("Not bipartite")
    else
        println("Bipartite")
        if length(graph) <= 200
            printbipartite(colors)
        end
    end
end

testisbipartite (generic function with 1 method)

In [99]:
for directed in ['u', 'd']
    for filetype in ['a', 'b']
        for i in 1:6
            filename = string("aod_testy1/4/",directed,"4",filetype,"-",i,".txt")
            testisbipartite(filename)
        end
    end
end

filename: aod_testy1/4/u4a-1.txt size: 15
Bipartite
Color: 0
2 3 8 9 10 11 12 13 14 15 
Color: 1
1 4 5 6 7 
filename: aod_testy1/4/u4a-2.txt size: 127
Bipartite
Color: 0
2 3 8 9 10 11 12 13 14 15 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 
Color: 1
1 4 5 6 7 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 
filename: aod_testy1/4/u4a-3.txt size: 1023
Bipartite
filename: aod_testy1/4/u4a-4.txt size: 16383
Bipartite
filename: aod_testy1/4/u4a-5.txt size: 131071
Bipartite
filename: aod_testy1/4/u4a-6.txt size: 1048575
Bipartite
filename: aod_testy1/4/u4b-1.txt size: 15
Not bipartite
filename: aod_testy1/4/u4b-2.txt size: 127
Not bipartite
filename: aod_testy1/4/u4b-3.txt size: 1023
Not bipartite
filename: aod_testy1/4/u4b